In [ ]:
import psycopg2
import csv
import os

# अपने डेटाबेस क्रेडेंशियल यहाँ बदलें (या पर्यावरण चर का उपयोग करें)
db_host = os.environ.get("DB_HOST", "dpg-d0bja2h5pdvs73cnu71g-a")
db_port = os.environ.get("DB_PORT", "5432")
db_name = os.environ.get("DB_NAME", "vitya_database")
db_user = os.environ.get("DB_USER", "vitya_database_user")
db_password = os.environ.get("DB_PASSWORD", "hUyT7WFfIPUZSCkUmYns2f8312pCMfP3")

tsv_file_path = r"C:\Users\preet\Desktop\VITYA-AI\tier3_city_person_expenses_combined .csv"  # अपनी TSV फ़ाइल का पाथ यहाँ दें
table_name = "expense"

conn = None

try:
    conn = psycopg2.connect(host=db_host, port=db_port, database=db_name, user=db_user, password=db_password)
    cur = conn.cursor()

    with open(tsv_file_path, 'r') as file:
        reader = csv.reader(file, delimiter='\t')  # डेलिमिटर के रूप में '\t' निर्दिष्ट करें
        header = next(reader)
        if header != ['amount', 'category', 'description', 'date', 'payment_type', 'user_id']:
            print("TSV फ़ाइल में अपेक्षित कॉलम नहीं हैं।")
            exit()

        insert_query = f"INSERT INTO {table_name} (amount, category, description, date, payment_type, user_id) VALUES (%s, %s, %s, %s, %s, %s)"
        for row in reader:
            cur.execute(insert_query, row)

    conn.commit()
    print(f"TSV फ़ाइल '{tsv_file_path}' से डेटा सफलतापूर्वक '{table_name}' टेबल में अपलोड किया गया!")

except psycopg2.Error as e:
    print(f"डेटा अपलोड करने में त्रुटि: {e}")
    if conn:
        conn.rollback()

except FileNotFoundError:
    print(f"फ़ाइल '{tsv_file_path}' नहीं मिली!")

except Exception as e:
    print(f"एक अप्रत्याशित त्रुटि हुई: {e}")

finally:
    if conn:
        cur.close()
        conn.close()
        print("डेटाबेस कनेक्शन बंद कर दिया गया।")